In [2]:
import os
import ipywidgets as widgets
from IPython.display import display

# configuration
video_dir = './input_videos'
tracking_data_dir = './tracking_data'
output_video_dir = './output_videos'

# Ensure all directories exist
dirs_to_check = [video_dir, tracking_data_dir, output_video_dir]
for dir in dirs_to_check:
    if not os.path.exists(dir):
        os.makedirs(dir)
        print(f"Directory created: {dir}")

# Function to select a video using an interactive widget
def interactive_select_video(video_dir):
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mov', '.MOV', '.mp4'))]
    if not video_files:
        print("No video files found in the directory.")
        return None
    video_selector = widgets.Dropdown(
        options=video_files,
        description='Select Video:',
        disabled=False,
    )
    display(video_selector)
    return video_selector

# Use the interactive selection component to select a video
video_selector = interactive_select_video(video_dir)


Dropdown(description='Select Video:', options=('20220520_150630.mp4', 'test_squat.mp4', 'IMG_9291.MOV'), value…

In [3]:

from tracking.video_processing_pipeline import process_video, body_parts

# Track the selected video
try:
    selected_video = video_selector.value
except AttributeError:
    print("No video was selected.")
    selected_video = None

if selected_video:
    # Process the selected video
    process_video(selected_video, video_dir, tracking_data_dir)

    # set the path to the tracking data file
    tracking_data_file = f'Tracking_{selected_video[:-4]}.csv'

    # Read and display a preview of the tracking data CSV
    tracking_data_df = pd.read_csv(os.path.join(tracking_data_dir, tracking_data_file))
    display(tracking_data_df.head())

/Users/linus/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
INFO:root:Processing video: IMG_9291.MOV
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO:root:Tracking data for IMG_9291.MOV stored in ./tracking_data/Tracking_IMG_9291.csv


,Label,Timestamp,Nose_x,Nose_y,Nose_z,Left eye inner_x,Left eye inner_y,Left eye inner_z,Left eye_x,Left eye_y,...,Left heel_z,Right heel_x,Right heel_y,Right heel_z,Left foot index_x,Left foot index_y,Left foot index_z,Right foot index_x,Right foot index_y,Right foot index_z
0,NaN,0,0.575357,0.187483,0.171514,0.563391,0.175160,0.103130,0.554227,0.176021,...,0.369710,0.703711,0.825419,0.323697,0.412216,0.768335,0.339353,0.774307,0.772267,0.277821
1,NaN,1,0.581328,0.187523,0.138201,0.568326,0.175089,0.082573,0.559319,0.175856,...,0.483672,0.703707,0.826693,0.417648,0.415525,0.768392,0.439838,0.774399,0.773887,0.360567
2,NaN,2,0.585067,0.186173,0.124684,0.572213,0.173227,0.072762,0.563482,0.173881,...,0.515734,0.703685,0.827518,0.436510,0.419109,0.767062,0.477188,0.774165,0.774068,0.382537
3,NaN,3,0.590993,0.185350,0.097543,0.578121,0.171792,0.049851,0.570002,0.172259,...,0.555306,0.703608,0.828397,0.452695,0.422472,0.767320,0.511560,0.774902,0.775044,0.393874
4,NaN,4,0.597038,0.185793,0.087584,0.583769,0.171815,0.042280,0.576236,0.172332,...,0.562781,0.703365,0.829192,0.455503,0.433097,0.766015,0.520593,0.775416,0.775771,0.397290


In [4]:
# Generate Video Tracking Preview
import pandas as pd
import os
import cv2
import mediapipe as mp
from IPython.display import Video, HTML

# Function to save video with landmarks from tracking data
def save_landmarks_on_video(video_path, output_video_path, tracking_data_path):
    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get the width, height, and frame rate of the video frame
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Read the tracking data from CSV
    tracking_data = pd.read_csv(tracking_data_path)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'avc1')  # or 'XVID'
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Process video and draw landmarks from tracking data
    for index, row in tracking_data.iterrows():
        # Read frame from video
        ret, frame = cap.read()
        if not ret:
            break

        # Draw landmarks on the frame
        for landmark in mp.solutions.pose.PoseLandmark:
            landmark_name = landmark.name.lower().replace('_', ' ')  # Convert to lowercase and replace underscores with spaces
            landmark_name = landmark_name.capitalize()  # Capitalize only the first letter of the first word
            landmark_x = f'{landmark_name}_x'
            landmark_y = f'{landmark_name}_y'
            if landmark_x in tracking_data.columns and landmark_y in tracking_data.columns and not pd.isna(row[landmark_x]) and not pd.isna(row[landmark_y]):
                x = int(row[landmark_x] * width)
                y = int(row[landmark_y] * height)
                # Check if the current landmark is one of the hips
                if landmark in [mp.solutions.pose.PoseLandmark.LEFT_HIP, mp.solutions.pose.PoseLandmark.RIGHT_HIP]:
                    # Highlight hips in red
                    cv2.circle(frame, (x, y), 15, (0, 0, 255), -1)
                else:
                    # Draw other landmarks in green
                    cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)


        # Write the frame with landmarks to the output video file
        out.write(frame)

    # Release everything when job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Video saved to {output_video_path}")

# Call the function to save the video with landmarks
video_path = os.path.join(video_dir, selected_video)

# Prepare the output video path
video_name = os.path.basename(video_path)
output_video_name = os.path.splitext(video_name)[0] + "_landmarks.mp4"
output_video_path = os.path.join(output_video_dir, output_video_name)
tracking_data_path = os.path.join(tracking_data_dir, tracking_data_file)
save_landmarks_on_video(video_path, output_video_path, tracking_data_path)

# Output
# Create an HTML string with the video embedded
video_html = f'''
Video saved to 
<a href='{output_video_path}' target='_blank'>{output_video_path}</a> <br><br>
<video height="500" controls allowfullscreen>
  <source src="{output_video_path}" type="video/mp4">
Your browser does not support the video tag.
</video>
'''
display(HTML(video_html))

Video saved to ./output_videos/IMG_9291_landmarks.mp4


In [25]:
# Interactive Plot with lables
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# uncomment to use a different video than the one used above
video_path = tracking_data_path

# Usage
csv_file_path = tracking_data_path
ignore_start = 0  # Number of frames to ignore at the start
ignore_end = 0  # Number of frames to ignore at the end

def plot_movement_data(csv_file_path, ignore_start=0, ignore_end=0, title="Average Hip Height over Time"):
    # Read the data from the CSV file
    data = pd.read_csv(csv_file_path, delimiter=",")

    # Ignore specified amount of frames at the beginning and/or end
    data = data[ignore_start : -ignore_end or None]

    # Calculate the average hip height
    data["Average_Hip_Height"] = (data["Left hip_y"] + data["Right hip_y"]) / 2

    # Create a scatter plot
    fig = go.Figure()

    # Check if 'Label' column exists and plot accordingly
    if 'Label' in data.columns and data['Label'].notna().any():
        # Define custom color mapping for movement types
        custom_color_mapping = {
            "Pause": "orange",
            "Ascending": "green",
            "Descending": "red",
            "Transition": "yellow",
            "Unknown": "grey",
        }

        for label, color in custom_color_mapping.items():
            mask = data["Label"] == label
            fig.add_trace(
                go.Scatter(
                    x=data.loc[mask, "Timestamp"],
                    y=data.loc[mask, "Average_Hip_Height"],
                    mode="markers",
                    name=label,
                    marker_color=color,
                    hovertemplate="x: %{x}",
                )
            )
    else:
        # Plot the average hip height over time in blue if no 'Label' column
        fig.add_trace(
            go.Scatter(
                x=data["Timestamp"],
                y=data["Average_Hip_Height"],
                mode="markers",
                name="Average Hip Height",
                marker_color="blue",
                hovertemplate="x: %{x}",
            )
        )

    # Get the x values of the first and last data points
    first_x = data["Timestamp"].iloc[0]
    last_x = data["Timestamp"].iloc[-1]

    # Add vertical lines at the first and last data points
    fig.add_shape(
        type="line",
        x0=first_x,
        y0=0,
        x1=first_x,
        y1=1,
        yref="paper",
        line=dict(color="Green", width=2),
    )
    fig.add_shape(
        type="line",
        x0=last_x,
        y0=0,
        x1=last_x,
        y1=1,
        yref="paper",
        line=dict(color="Red", width=2),
    )

    # Add annotations for the first and last data points
    fig.add_annotation(
        x=first_x, y=0.05, text=f"Start: {first_x}", showarrow=False, yref="paper"
    )
    fig.add_annotation(
        x=last_x, y=0.05, text=f"End: {last_x}", showarrow=False, yref="paper"
    )

    # Increase the number of grid lines
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="LightGrey")
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="LightGrey", autorange="reversed")

    # Remove the light blue background
    fig.update_layout(
        width=1000,  # Width of the figure in pixels
        height=600,  # Height of the figure in pixels
        hovermode="x", # Hovermode for the cursor
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        title=f"{title} - {os.path.basename(csv_file_path)}"  # Add a title to the plot
    )

    fig.show()

# Call the function to plot the data
plot_movement_data(csv_file_path, ignore_start, ignore_end, title="Average Hip Height over Time")

# Add another plot with a different headline and file path below
# Usage
csv_file_path = "../../data/training/labled_data/Tracking_video01_labled.csv"  # Use a different file path
ignore_start = 0  # Number of frames to ignore at the start
ignore_end = 0  # Number of frames to ignore at the end

# Call the function to plot the data
plot_movement_data(csv_file_path, ignore_start, ignore_end, title="Labled Example Data")